In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot

In [2]:
fraud = pd.read_csv("Fraud_check.csv")

In [3]:
fraud

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [4]:
##Converting the Taxable income variable to bucketing. 
fraud["income"]="<=30000"
fraud.loc[fraud["Taxable.Income"]>=30000,"income"]="Good"
fraud.loc[fraud["Taxable.Income"]<=30000,"income"]="Risky"

In [5]:
##Droping the Taxable income variable
fraud.drop(["Taxable.Income"],axis=1,inplace=True)

In [6]:
fraud.rename(columns={"Undergrad":"undergrad","Marital.Status":"marital","City.Population":"population","Work.Experience":"experience","Urban":"urban"},inplace=True)
## As we are getting error as "ValueError: could not convert string to float: 'YES'".
## Model.fit doesnt not consider String. So, we encode

In [7]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
for column_name in fraud.columns:
    if fraud[column_name].dtype == object:
        fraud[column_name] = le.fit_transform(fraud[column_name])
    else:
        pass

In [8]:
##Splitting the data into featuers and labels
features = fraud.iloc[:,0:5]
labels = fraud.iloc[:,5]

In [9]:
## Collecting the column names
colnames = list(fraud.columns)
predictors = colnames[0:5]
target = colnames[5]
##Splitting the data into train and test

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(features,labels,test_size = 0.2,stratify = labels)

In [11]:
##Model building
from sklearn.ensemble import RandomForestClassifier as RF
model = RF(n_jobs = 3,n_estimators = 15, oob_score = True, criterion = "entropy")
model.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=15, n_jobs=3,
                       oob_score=True)

In [12]:
model.estimators_
model.classes_
model.n_features_
model.n_classes_

2

In [13]:
model.n_outputs_

1

In [14]:
model.oob_score_

0.7291666666666666

In [15]:
##Predictions on train data
prediction = model.predict(x_train)

In [16]:
##Accuracy
# For accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_train,prediction)

In [17]:
np.mean(prediction == y_train)

0.9875

In [18]:
##Confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_train,prediction)

In [19]:
##Prediction on test data
pred_test = model.predict(x_test)

In [20]:
##Accuracy
acc_test =accuracy_score(y_test,pred_test)

In [21]:
## In random forest we can plot a Decision tree present in Random forest
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.externals.six import StringIO

ModuleNotFoundError: No module named 'pydotplus'

In [22]:
tree = model.estimators_[5]

In [23]:
dot_data = StringIO()
export_graphviz(tree,out_file = dot_data, filled = True,rounded = True, feature_names = predictors ,class_names = target,impurity =False)

NameError: name 'StringIO' is not defined

In [24]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

NameError: name 'pydotplus' is not defined

In [25]:
## Creating pdf and png file the selected decision tree
graph.write_pdf('fraudrf.pdf')
graph.write_png('fraudrf.png')

NameError: name 'graph' is not defined

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Bagged Decision Trees for Classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
filename = 'Fraud_check.csv'
names = ["Undergrad","Marital.Status","Taxable.Income","City.Population","Work.Experience","Urban"]
dataframe = read_csv(filename, names=names)
dataframe

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
1,NO,Single,68833,50047,10,YES
2,YES,Divorced,33700,134075,18,YES
3,NO,Married,36925,160205,30,YES
4,YES,Single,50190,193264,15,YES
...,...,...,...,...,...,...
596,YES,Divorced,76340,39492,7,YES
597,YES,Divorced,69967,55369,2,YES
598,NO,Divorced,47334,154058,0,YES
599,YES,Married,98592,180083,17,NO


In [3]:
#importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv("Fraud_check.csv")

In [4]:
#Viewing top 5 rows of dataframe
df.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [5]:
#Creating dummy vairables for ['Undergrad','Marital.Status','Urban'] dropping first dummy variable
df=pd.get_dummies(df,columns=['Undergrad','Marital.Status','Urban'], drop_first=True)

In [6]:
#Creating new cols TaxInc and dividing 'Taxable.Income' cols on the basis of [10002,30000,99620] for Risky and Good
df["TaxInc"] = pd.cut(df["Taxable.Income"], bins = [10002,30000,99620], labels = ["Risky", "Good"])

In [7]:
#After creation of new col. TaxInc also made its dummies var concating right side of df
df = pd.get_dummies(df,columns = ["TaxInc"],drop_first=True)

In [8]:
#Viewing buttom 10 observations
df.tail(10)

,Taxable.Income,City.Population,Work.Experience,Undergrad_YES,Marital.Status_Married,Marital.Status_Single,Urban_YES,TaxInc_Good
590,43018,85195,14,0,1,0,1,1
591,27394,132859,18,1,0,1,1,0
592,68152,75143,16,1,0,1,0,1
593,84775,131963,10,0,0,0,1,1
594,47364,97526,9,0,1,0,1,1
595,76340,39492,7,1,0,0,1,1
596,69967,55369,2,1,0,0,1,1
597,47334,154058,0,0,0,0,1,1
598,98592,180083,17,1,1,0,0,1
599,96519,158137,16,0,0,0,0,1


In [16]:
array = dataframe.values
X = array[:,0:5]
Y = array[:,5]
seed = 7

kfold = KFold(n_splits=10, )
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, )
results = cross_val_score(model, X, Y, cv=kfold)


In [11]:
results

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [18]:
print(results.mean())

nan


In [9]:
# Normalization function 
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

In [10]:
# Normalized data frame (considering the numerical part of data)
df_norm = norm_func(df.iloc[:,1:])
df_norm.tail(10)

,City.Population,Work.Experience,Undergrad_YES,Marital.Status_Married,Marital.Status_Single,Urban_YES,TaxInc_Good
590,0.341473,0.466667,0.0,1.0,0.0,1.0,1.0
591,0.615406,0.600000,1.0,0.0,1.0,1.0,0.0
592,0.283703,0.533333,1.0,0.0,1.0,0.0,1.0
593,0.610256,0.333333,0.0,0.0,0.0,1.0,1.0
594,0.412341,0.300000,0.0,1.0,0.0,1.0,1.0
595,0.078811,0.233333,1.0,0.0,0.0,1.0,1.0
596,0.170058,0.066667,1.0,0.0,0.0,1.0,1.0
597,0.737240,0.000000,0.0,0.0,0.0,1.0,1.0
598,0.886810,0.566667,1.0,1.0,0.0,0.0,1.0
599,0.760683,0.533333,0.0,0.0,0.0,0.0,1.0


In [11]:
array = dataframe.values
X = array[:,0:5]
Y = array[:,5]
seed = 7

kfold = KFold(n_splits=10, )
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, )
results = cross_val_score(model, X, Y, cv=kfold)


In [12]:
results

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [29]:
# Declaring features & target
X = df_norm.drop(['TaxInc_Good'], axis=1)
y = df_norm['TaxInc_Good']

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
# Splitting data into train & test
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=0)

In [32]:
#Choosing RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [33]:
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

RandomForestClassifier(n_estimators=200)

In [38]:
print('Train accuracy: {}'.format(fraud.score(Xtrain, ytrain)))
print('Test accuracy: {}'.format(forest.score(Xtest, ytest)))

NameError: name 'fraud' is not defined

In [18]:
#Complete Iris dataset
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
Fraud['Urban']= label_encoder.fit_transform(Fraud['Urban']) 

TypeError: string indices must be integers

In [ ]:
x=Fraud.iloc[:,0:4]
y=Fraud['Urban']

In [39]:
#Complete Iris dataset
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
Fraud['Undergrad']= label_encoder.fit_transform(Fraud['Undergrad']) 
Fraud['Marital.Status']= label_encoder.fit_transform(Fraud['Marital.Status']) 
Fraud['Urban']= label_encoder.fit_transform(Fraud['Urban']) 

TypeError: string indices must be integers

In [43]:
array = dataframe.values
X = array[:,0:5]
Y = array[:,5]
seed = 7

kfold = KFold(n_splits=10, )
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, )
results = cross_val_score(model, X, Y, cv=kfold)


In [44]:
results

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [45]:
print(results.mean())

nan


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing # SES
from statsmodels.tsa.holtwinters import Holt # Holts Exponential Smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing # 
import statsmodels.graphics.tsaplots as tsa_plots
import statsmodels.tsa.statespace as tm_models
from datetime import datetime,time
#from sm.tsa.statespace import sa
Fraud = pd.read_csv('Fraud_check.csv')


In [2]:
Fraud

,Undergrad,MaritalStatus,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [7]:
Fraud.rename(columns={" ('000)":"Taxable.Income"},inplace=True)   
# Converting the normal index of Amtrak to time stamp 
Fraud.index = pd.to_datetime(Fraud.MaritalStatus, format='%b-%y')

ValueError: time data 'Single' does not match format '%b-%y' (match)